In [5]:
using SwiftObjectStores
using ColoringNames
using Distributions
using MLDataUtils
using Iterators
using MLLabelUtils
using StaticArrays
using Juno
using StatsBase
using Colors
using DataFrames
using Query
using Plots
gr()

const od =(MLDataUtils.ObsDim.First(), MLDataUtils.ObsDim.Last())
const serv=SwiftService()

const valid_raw = get_file(fh->readdlm(fh,'\t'), serv, "color", "monroe/dev.csv")
const valid_hsv, valid_terms_padded, encoding = prepare_data(valid_raw; do_demacate=false)
const valid_text = valid_raw[:, 1]

const train_raw = get_file(fh->readdlm(fh,'\t'), serv, "color", "monroe/train.csv")
const train_hsv, train_terms_padded, encoding = prepare_data(train_raw, encoding; do_demacate=false)
const train_text = train_raw[:, 1]




1523108-element Array{Any,1}:
 "acid green"
 "acid green"
 "acid green"
 "acid green"
 "acid green"
 "acid green"
 "acid green"
 "acid green"
 "acid green"
 "acid green"
 "acid green"
 "acid green"
 "acid green"
 ⋮           
 "yuck"      
 "yuck"      
 "yuck"      
 "yuck"      
 "yuck"      
 "yuck"      
 "yuck"      
 "yuck"      
 "yuck"      
 "yuck"      
 "yuck"      
 "yuck"      

In [6]:


function pairwise_stats(fun, names, hsvs)
    dt = DataFrame(name=String[], n_samples=Int[], hs=Float64[], hv=Float64[], vs=Float64[])
    @progress for (name, inds) in labelmap(names)
        eg_hsvs = @view hsvs[inds, :]

        hs = fun(eg_hsvs[:,1], eg_hsvs[:,2])
        hv = fun(eg_hsvs[:,1], eg_hsvs[:,3])
        vs = fun(eg_hsvs[:,3], eg_hsvs[:,2])

        push!(dt, [name, length(inds), hs, hv, vs])
    end
    dt
end
spearman = pairwise_stats(corspearman, train_text, train_hsv)



describe(spearman)

name
Summary Stats:
Length:         829
Type:           String
Number Unique:  829

n_samples
Summary Stats:
Mean:           1837.283474
Minimum:        70.000000
1st Quartile:   109.000000
Median:         214.000000
3rd Quartile:   627.000000
Maximum:        152953.000000
Length:         829
Type:           Int64

hs
Summary Stats:
Mean:           0.022951
Minimum:        -0.429653
1st Quartile:   -0.086345
Median:         0.019934
3rd Quartile:   0.121130
Maximum:        0.577041
Length:         829
Type:           Float64

hv
Summary Stats:
Mean:           0.000171
Minimum:        -0.577392
1st Quartile:   -0.101860
Median:         0.003642
3rd Quartile:   0.103550
Maximum:        0.497044
Length:         829
Type:           Float64

vs
Summary Stats:
Mean:           -0.048629
Minimum:        -0.497048
1st Quartile:   -0.132881
Median:         -0.050129
3rd Quartile:   0.031004
Maximum:        0.455614
Length:         829
Type:           Float64



In [7]:
histogram(([spearman[:hs],
            spearman[:hv],
            spearman[:vs],
            maximum(abs.(Matrix(spearman[[:hs, :hv, :vs]])), 2)[:]]),
            layout=(4,1),
            legend=false
)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.25 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 0 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.4 
 
 
 -0.2 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0.4 
 
 
 0.5 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 60

In [ ]:

high_cor =
    @from r in spearman begin
    @where abs(r.hs) > 0.30 || abs(r.hv) > 0.30 || abs(r.vs) > 0.30
    @orderby abs(r.hs) + abs(r.hv) + abs(r.vs)
    @select r
    @collect DataFrame
end
